In [1]:
%load_ext autoreload
%autoreload 2

import stac_mjx 
from pathlib import Path

import numpy as np
        
# Choose parent directory as base path to make relative pathing easier 
base_path = Path.cwd().parent
stac_config_path = base_path / "configs/stac_mouse.yaml"
model_config_path = base_path / "configs/mouse.yaml"

stac_cfg, model_cfg = stac_mjx.load_configs(stac_config_path, model_config_path)

data_path = base_path / stac_cfg.data_path
kp_data, sorted_kp_names = stac_mjx.load_data(data_path, model_cfg)
print("kp_data shape", kp_data.shape)

def get_feet_pts(kp_data, frame):
    feet = ["Forepaw_R", "Forepaw_L", "Lisfranc_L", "Lisfranc_R"]
    feetPts = np.empty((0,3))

    for foot in feet:
        idx = 3*sorted_kp_names.index(foot)
        #print("Foot:", foot)
        #print("idx: ", idx)
        
        #print(kp_data[frame, idx:idx+3])
        feetPts = np.append(feetPts, [kp_data[frame, idx:idx+3]], axis = 0)

    return feetPts

feetpts = get_feet_pts(kp_data, 1)
print("feetpts")
print(feetpts)

kp_data shape (3600, 102)
feetpts
[[0.07909218 0.02146954 0.37473813]
 [0.10951491 0.023065   0.36979905]
 [0.07752494 0.01065001 0.44042069]
 [0.052271   0.01526332 0.41534567]]


In [2]:

def plane_normal_and_height(points):
    """
    Calculate the normal vector and the height of the plane at the barycenter.

    Parameters:
    points (numpy.ndarray): A (4, 3) array where each row is a 3D point.

    Returns:
    normal (numpy.ndarray): The normal vector of the plane.
    height (float): The height of the plane at the barycenter.
    barycenter (numpy.ndarray): The barycenter of the four points.
    """
    assert points.shape == (4, 3), "Input must be a (4, 3) array representing four 3D points."

    # Calculate vectors from point 0 to point 1 and point 0 to point 2
    v1 = points[1] - points[0]
    v2 = points[2] - points[0]

    # Calculate the normal vector using the cross product
    normal = np.cross(v1, v2)
    normal = normal / np.linalg.norm(normal)  # Normalize the normal vector

    # Calculate the barycenter (centroid) of the four points
    barycenter = np.mean(points, axis=0)

    # Calculate the height of the plane at the barycenter
    # Height is the distance from the origin to the plane along the normal vector
    # The plane equation is: normal.dot(x) = d
    # where x is any point on the plane
    d = np.dot(normal, points[0])
    height = np.dot(normal, barycenter) - d

    return normal, height, barycenter

norm0, _, _ = plane_normal_and_height(feetpts) 
print(norm0)



[ 0.02545142 -0.98648051 -0.16189018]


In [3]:
num_frames = kp_data.shape[0]
normals = np.zeros((num_frames, 3))

for i in range(num_frames):
    feetPts = get_feet_pts(kp_data, i)
    normal, _, _ = plane_normal_and_height(feetPts)
    normals[i] = normal

average_normal = np.mean(normals, axis=0)
print("avg normal: ", average_normal)

avg normal:  [ 0.00385561 -0.96136723 -0.16659539]


In [4]:
def rotate_normal_to_z(normal):
    """
    Rotate a normal vector to align with the z-axis.
    
    Parameters:
    normal (numpy.ndarray): A 3D normal vector.
    
    Returns:
    rotation_matrix (numpy.ndarray): The 3x3 rotation matrix that aligns the normal to the z-axis.
    """
    # Normalize the normal vector
    normal = normal / np.linalg.norm(normal)
    
    # Target vector (z-axis)
    z_axis = np.array([0, 0, 1])
    
    # Calculate the rotation axis (cross product)
    axis = np.cross(normal, z_axis)
    
    # Calculate the angle between the normal and the z-axis
    angle = np.arccos(np.dot(normal, z_axis))
    
    # Special case: if the normal is already aligned with the z-axis
    if np.allclose(angle, 0):
        return np.eye(3)
    
    # Skew-symmetric matrix for the rotation axis
    K = np.array([[0, -axis[2], axis[1]],
                  [axis[2], 0, -axis[0]],
                  [-axis[1], axis[0], 0]])
    
    # Rodrigues' rotation formula
    rotation_matrix = np.eye(3) + np.sin(angle) * K + (1 - np.cos(angle)) * np.dot(K, K)
    
    return rotation_matrix

def plane_normal_from_points(points):
    """
    Calculate the normal vector of the plane defined by four points.
    
    Parameters:
    points (numpy.ndarray): A (4, 3) array where each row is a 3D point.
    
    Returns:
    normal (numpy.ndarray): The normal vector of the plane.
    """
    v1 = points[1] - points[0]
    v2 = points[2] - points[0]
    normal = np.cross(v1, v2)
    normal = normal / np.linalg.norm(normal)  # Normalize the normal vector
    return normal

def rotate_points(points, rotation_matrix):
    """
    Rotate a set of points using the given rotation matrix.
    
    Parameters:
    points (numpy.ndarray): A 2D array of shape (num_frames, num_points * 3).
    rotation_matrix (numpy.ndarray): A 3x3 rotation matrix.
    
    Returns:
    rotated_points (numpy.ndarray): The rotated points of shape (num_frames, num_points * 3).
    """
    num_frames, num_points_3 = points.shape
    num_points = num_points_3 // 3
    
    # Reshape to (num_frames, num_points, 3)
    points_reshaped = points.reshape(num_frames, num_points, 3)
    
    # Rotate each point
    rotated_points_reshaped = np.dot(points_reshaped, rotation_matrix.T)
    
    # Reshape back to (num_frames, num_points * 3)
    rotated_points = rotated_points_reshaped.reshape(num_frames, num_points_3)
    
    return rotated_points

normal = plane_normal_from_points(feetPts)
print("normal:", normal)

# Calculate the rotation matrix to align the normal vector with the z-axis
rotation_matrix = rotate_normal_to_z(normal)

rotated_points = rotate_points(kp_data, rotation_matrix)

normal: [-0.14091238 -0.85242854 -0.50349705]
